In [2]:
import requests
import urllib3
import json
import re
import ssl
import socket
import warnings
import base64
from typing import Dict, List, Any, Optional
from urllib.parse import urlparse
import sslyze
import cryptography
from cryptography import x509
from cryptography.hazmat.backends import default_backend

class APISecurity:
    def __init__(self, base_url: str, headers: Optional[Dict[str, str]] = None):
        """
        Initialize API Security Scanner

        :param base_url: Base URL of the API to be tested
        :param headers: Optional headers to be used in requests
        """
        self.base_url = base_url
        self.headers = headers or {}
        self.vulnerabilities = []

        # Disable SSL warnings for testing
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

    def test_authentication(self, endpoints: List[str], auth_types: List[str] = None):
        """
        Test authentication mechanisms for given endpoints

        :param endpoints: List of API endpoints to test
        :param auth_types: Types of authentication to check (basic, bearer, etc.)
        """
        auth_types = auth_types or ['basic', 'bearer', 'no_auth']

        for endpoint in endpoints:
            full_url = f"{self.base_url.rstrip('/')}/{endpoint.lstrip('/')}"

            # Test no authentication
            if 'no_auth' in auth_types:
                try:
                    response = requests.get(full_url, verify=False)
                    if response.status_code < 401:
                        self.vulnerabilities.append({
                            'type': 'Authentication',
                            'severity': 'High',
                            'endpoint': endpoint,
                            'description': 'Endpoint accessible without authentication'
                        })
                except Exception as e:
                    print(f"Error testing {endpoint}: {e}")

            # Test basic authentication bypass
            if 'basic' in auth_types:
                try:
                    # Try empty credentials
                    response = requests.get(full_url, auth=('', ''), verify=False)
                    if response.status_code < 401:
                        self.vulnerabilities.append({
                            'type': 'Authentication',
                            'severity': 'Medium',
                            'endpoint': endpoint,
                            'description': 'Possible basic authentication bypass'
                        })
                except Exception as e:
                    print(f"Error testing basic auth for {endpoint}: {e}")

    def check_https_encryption(self):
        """
        Check HTTPS encryption and SSL/TLS configuration
        """
        parsed_url = urlparse(self.base_url)
        hostname = parsed_url.hostname
        port = parsed_url.port or (443 if parsed_url.scheme == 'https' else 80)

        try:
            # Check if HTTPS is used
            if parsed_url.scheme != 'https':
                self.vulnerabilities.append({
                    'type': 'Transport Security',
                    'severity': 'High',
                    'description': 'API not using HTTPS'
                })

            # Perform deeper SSL/TLS analysis
            context = ssl.create_default_context()
            with socket.create_connection((hostname, port)) as sock:
                with context.wrap_socket(sock, server_hostname=hostname) as secure_sock:
                    cert = secure_sock.getpeercert(binary_form=True)
                    x509_cert = x509.load_der_x509_certificate(cert, default_backend())

                    # Check certificate validity
                    if x509_cert.not_valid_before > ssl.cert_time_to_seconds() or \
                       x509_cert.not_valid_after < ssl.cert_time_to_seconds():
                        self.vulnerabilities.append({
                            'type': 'SSL/TLS',
                            'severity': 'Medium',
                            'description': 'Invalid or expired SSL certificate'
                        })

        except Exception as e:
            self.vulnerabilities.append({
                'type': 'Transport Security',
                'severity': 'High',
                'description': f'SSL/TLS connection error: {str(e)}'
            })

    def detect_sensitive_data_exposure(self, endpoints: List[str]):
        """
        Detect potential sensitive data exposure in API responses

        :param endpoints: List of API endpoints to test
        """
        sensitive_patterns = {
            'email': r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',
            'credit_card': r'\b(?:\d{4}[-\s]?){3}\d{4}\b',
            'ssn': r'\b\d{3}-\d{2}-\d{4}\b',
            'api_key': r'[A-Za-z0-9]{32,}'
        }
        for endpoint in endpoints:
            full_url = f"{self.base_url.rstrip('/')}/{endpoint.lstrip('/')}"

            try:
                response = requests.get(full_url, headers=self.headers, verify=False)

                # Check response body
                body_text = response.text
                for pattern_name, pattern in sensitive_patterns.items():
                    matches = re.findall(pattern, body_text)
                    if matches:
                        self.vulnerabilities.append({
                            'type': 'Data Exposure',
                            'severity': 'High',
                            'endpoint': endpoint,
                            'description': f'Potential {pattern_name} exposure',
                            'matches': matches
                        })

                # Check headers
                for header, value in response.headers.items():
                    for pattern_name, pattern in sensitive_patterns.items():
                        matches = re.findall(pattern, str(value))
                        if matches:
                            self.vulnerabilities.append({
                                'type': 'Header Exposure',
                                'severity': 'Medium',
                                'endpoint': endpoint,
                                'header': header,
                                'description': f'Potential {pattern_name} exposure in header'
                            })

            except Exception as e:
                print(f"Error scanning {endpoint}: {e}")

    def generate_security_report(self) -> Dict[str, Any]:
        """
        Generate a comprehensive security report

        :return: Dictionary containing vulnerability details
        """
        # Aggregate and categorize vulnerabilities
        report = {
            'total_vulnerabilities': len(self.vulnerabilities),
            'severity_breakdown': {
                'High': 0,
                'Medium': 0,
                'Low': 0
            },
            'vulnerabilities': self.vulnerabilities
        }

        # Count vulnerabilities by severity
        for vuln in self.vulnerabilities:
            report['severity_breakdown'][vuln.get('severity', 'Low')] += 1

        return report

def main():
    # Example usage
    api_scanner = APISecurity(
        base_url='https://jsonplaceholder.typicode.com/',
        headers={'User-Agent': 'API Security Scanner'}
    )

    # Define test endpoints
    test_endpoints = [
        'users',
        'profile',
        'admin',
        'sensitive-data'
    ]

    # Perform security tests
    api_scanner.check_https_encryption()
    api_scanner.test_authentication(test_endpoints)
    api_scanner.detect_sensitive_data_exposure(test_endpoints)

    # Generate and print security report
    report = api_scanner.generate_security_report()

    print("API Security Scan Report")
    print("=" * 30)
    print(f"Total Vulnerabilities: {report['total_vulnerabilities']}")
    print("\nSeverity Breakdown:")
    for severity, count in report['severity_breakdown'].items():
        print(f"{severity} Risk: {count}")

    print("\nDetailed Vulnerabilities:")
    for vuln in report['vulnerabilities']:
        print(f"- {vuln['type']} ({vuln['severity']}): {vuln['description']}")

    # Optional: Save report to JSON
    with open('api_security_report.json', 'w') as f:
        json.dump(report, f, indent=2)

if __name__ == "__main__":
    main()

C:\Users\Ash\AppData\Local\Temp\ipykernel_44416\1160721855.py:97: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_before_utc.
  if x509_cert.not_valid_before > ssl.cert_time_to_seconds() or \


API Security Scan Report
Total Vulnerabilities: 8

Severity Breakdown:
High Risk: 3
Medium Risk: 5
Low Risk: 0

Detailed Vulnerabilities:
- Transport Security (High): SSL/TLS connection error: cert_time_to_seconds() missing 1 required positional argument: 'cert_time'
- Authentication (High): Endpoint accessible without authentication
- Authentication (Medium): Possible basic authentication bypass
- Data Exposure (High): Potential email exposure
- Header Exposure (Medium): Potential api_key exposure in header
- Header Exposure (Medium): Potential api_key exposure in header
- Header Exposure (Medium): Potential api_key exposure in header
- Header Exposure (Medium): Potential api_key exposure in header
